##### This file creates the final csv which can be upload to the moodle named as moodle_gradeA2.csv

In [ ]:
import os 
import pandas as pd
import numpy as np 
from pprint import pprint

In [ ]:
home_dir: str = os.getcwd()
logs_dir: str = os.path.join(home_dir, 'logs')
plot_dir: str = os.path.join(home_dir, 'plots')
sub_dir: str = os.path.join(home_dir, 'subA3')
info_task1_dir: str = os.path.join(home_dir, 'info_task1')
info_task2_dir: str = os.path.join(home_dir, 'info_task2') 
results_dir: str = os.path.join(home_dir, 'results')

print (home_dir, logs_dir, plot_dir, sub_dir, results_dir)

Scaling factors

In [ ]:
# 40% of correctness, 40% of performnace, and 20% of report
final_scaling = {
    'report': 1.8 / 100, 
    'correctness': 3.6 / 100, 
    'performance': 3.6 / 100,
}
final_scaling

Get kerberos from the entry number and also the group ids

In [ ]:
def get_kerberos(Entry_no: str) -> str:
    year, dept, endwith = Entry_no[2: 4], Entry_no[4: 7], Entry_no[7: ]
    kerberos = dept + year + endwith
    return kerberos.lower()

Read the pairs of students who are doing assignment in pair

In [ ]:
student_pair_df = pd.read_csv(os.path.join(home_dir, 'studentPairs.csv'))
# student_pair_df.set_index('studentA', inplace=True)

Update the weights of scan, since they are default /100 for each part

In [ ]:
weight_report = {
    'Table': 0.3,
    'Scan' : 0.2, 
    'Plots' : 0.2,
    'Iso-efficiency-and-Seq-Frac' : 0.3
} # Update these accordingly 

scan_df = pd.read_csv(os.path.join(home_dir, 'report_scores.csv'))
scan_df['Total Score'] = 0
for metric in weight_report.keys():
    scan_df['Total Score'] += scan_df[metric] * weight_report[metric]
# need_demo = scan_df[scan_df['Demo required'] == 'Need demo']['Entry Number']
# need_demo.to_csv(os.path.join(home_dir, 'need_demo.csv'))
scan_df

Make a format for comment to pass

In [ ]:
# penalty_df = pd.read_csv(os.path.join(home_dir, 'demo_penalty.csv'))
# penalty_df.set_index('email_address', inplace=True)

# demo_students = list(penalty_df.index)
# demo_students = [eno.split('@')[0] for eno in demo_students]

# def get_penalty(entry_no):
#     return 0.75 * penalty_df.loc[f'{entry_no}@cse.iitd.ac.in', 'penalty']

Reading all three type of scores and some renaming

In [ ]:
# 1. Read all three components
correct_df = pd.read_csv(os.path.join(results_dir, 'correctness.csv'))
performance_df = pd.read_csv(os.path.join(results_dir, 'runtime_score.csv'))

# 2. Some renaming for better readability
correct_df.rename(columns={'Total Score': 'TOTAL CORRECTNESS SCORE(out of 100)'}, inplace=True)
scan_df.rename(columns={'Total Score': 'TOTAL REPORT SCORE(out of 100)'}, inplace=True)
performance_df.rename(columns={'Total Score': 'TOTAL PERFORMANCE SCORE(out of 100)'}, inplace=True)

# correct_df.set_index('kerberos', inplace=True)
# performance_df.set_index('kerberos', inplace=True)

Filling information to the comment for the moodle

In [ ]:
# some renaming
# scan_df = scan_df.replace(np.nan, 'No')
# scan_df.drop(columns='Demo required ?', inplace=True)
comments = {k: {} for k in list(correct_df['kerberos'])}
print(correct_df.columns)

paired_to = dict()
for _, row in student_pair_df.iterrows():
    paired_to[row['studentA']] = row['group']
    paired_to[row['studentB']] = row['group']
    
# 1. Then find fill down details for correctness
for _, student in correct_df.iterrows():
    correctness_marks = dict()
    Entry_no = student['kerberos']
    for cols in correct_df.columns:
        if cols == 'Entry_no' or cols == 'kerberos':
            continue
        correctness_marks[cols] = student[cols]
    print (f'Entry no: {Entry_no}, marks: {correctness_marks}')
    comments[Entry_no]['correctness'] = correctness_marks
    
# 2. Scan marks
print (scan_df.columns)
for _, student in scan_df.iterrows():
    report_marks = dict() 
    Entry_no = get_kerberos(student['Entry_no'])
    for cols in scan_df.columns:
        if cols == 'Entry_no' or cols == 'kerberos':
            continue
        report_marks[cols] = student[cols]
    comments[Entry_no]['report'] = report_marks

# 3. Performance marks
print (performance_df.columns)
for _, student in performance_df.iterrows():
    performance_marks = dict() 
    Entry_no = student['kerberos']

    for cols in performance_df.columns:
        if cols == 'Entry_no' or cols == 'kerberos':
            continue
        performance_marks[cols] = student[cols] 
    comments[Entry_no]['performance'] = performance_marks

pprint(comments)

Prepare the final csv, which will be upload to moodle

In [ ]:
# Read the students who are active on moodle 
moodle_export_df = pd.read_csv(os.path.join(home_dir, 'col380exports.csv'))
# Final dataframe : grades going to publish to moodle, is filled to this.
moodle_csv = {'userid': [], 'marks': [], 'comments': []}

print (set(comments.keys()) - set(paired_to.keys()))

for Entry_no, comment in comments.items():
    Entry_no = Entry_no.lower()
    
    groupid = paired_to[Entry_no]
    groupid = groupid.lower()
    studentA, studentB = "", ""
    if '_' in groupid:
        studentA, studentB = groupid.split('_')
    else:
        studentA = studentB = groupid
    studentA = studentA.strip().lower()
    studentB = studentB.strip().lower()
    if Entry_no == 'cs1190452':
        print (f'Yes, {studentA}, {studentB} and {groupid}')
    
    print (f'StudentA = {studentA}, studentB = {studentB}') 

    penalty = 0.0
    # if (studentA in demo_students):
    #     penalty = get_penalty(studentA)
    # if (studentB in demo_students):
    #     penalty = get_penalty(studentB)
    # comment['penalty_for_demo'] = penalty

    # Final global scaling of marks 
    scaled_correctness_score = comment['correctness']['TOTAL CORRECTNESS SCORE(out of 100)'] * final_scaling['correctness']
    scaled_performance_score = comment['performance']['TOTAL PERFORMANCE SCORE(out of 100)'] * final_scaling['performance']
    scaled_report_score = comment['report']['TOTAL REPORT SCORE(out of 100)'] * final_scaling['report']

    if 'report' not in comment:
        print (f"Yes : {studentA}, {studentB}")
        comment['report'] = {}
        comment['report']['TOTAL REPORT SCORE(out of 100)'] = np.nan

    marks = scaled_report_score + max(0, scaled_correctness_score + scaled_performance_score - penalty)
    marks = round(marks, 2)
    if studentA == studentB:
        moodle_csv['userid'].append(studentA)
        moodle_csv['marks'].append(marks)
        moodle_csv['comments'].append(comment)
    else:
        moodle_csv['userid'].extend([studentA, studentB])
        moodle_csv['marks'].extend([marks, marks])
        moodle_csv['comments'].extend([comment, comment])


print(moodle_csv)    

alluserid = set(list(moodle_export_df['ID number']))
submitted = set(list(moodle_csv['userid']))
print (f'Total user on moodle avail {alluserid}, how many submitted = {submitted}')

notsubmitted = alluserid - submitted
dropcourse = submitted - alluserid 
commons = (dropcourse) and (notsubmitted)

pprint (f'len: {len(commons)}, Commons: {commons}')
pprint (f'len: {len(notsubmitted)} Not sumbitted: {notsubmitted}')
pprint (f'len: {len(dropcourse)} Dropped : {dropcourse}')

for e in notsubmitted:
    moodle_csv['userid'].append(e)
    moodle_csv['marks'].append(0)
    moodle_csv['comments'].append('Not Submitted')

moodle_csv = pd.DataFrame(moodle_csv)
moodle_csv.set_index('userid', inplace=True)

for e in dropcourse:
    moodle_csv.drop(e, inplace=True)

moodle_csv.to_csv(os.path.join(results_dir, 'gradesA3.csv'))

Stats for the final results

In [ ]:
stats_df = pd.read_csv(os.path.join(results_dir, 'gradesA3.csv'))
marks = stats_df[stats_df['comments'] != 'Not Submitted']['marks']
userids = stats_df[stats_df['comments'] != 'Not Submitted']['userid']
distribution = marks.describe()
print(distribution)
pprint (marks)

In [ ]:
topScores = pd.Series(marks.nlargest(18).unique())
print (len(topScores))
print (f'Stats for top 20: {topScores.describe()}')
topScores = pd.Series(marks.nlargest(37).unique())
print (len(topScores))
print (f'Stats for top 20: {topScores.describe()}')